<strong><b><font size="5">Data Science Academy</font></b></strong>

<strong><b><font size="5">Web Scraping e Análise de Dados</font></b></strong>

<strong><b><font size="5">Lab 10 - Projeto Final</font></b></strong>

<strong><b><font size="3">Web Scraping, Processamento de Linguagem Natural, Modelagem de Tópicos e Análise de Sentimentos em Discursos Políticos</font></b></strong>

![title](imagens/lab10.png)

## Definição do Problema

Leia o manual em pdf onde você encontrou este Jupyter Notebook.

## Fonte de Dados

As fontes de dados são os sites oficiais dos políticos que estamos extraindo os discursos:

Prefeito da Cidade de New York: https://www1.nyc.gov

Governador do Estado de New York: https://www.governor.ny.gov

## Carregando os Pacotes Usados Neste Jupyter Notebook

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook, se necessário.

In [ ]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Pacote para análise de sentimentos
# https://pypi.org/project/vaderSentiment/
!pip install -q vaderSentiment

In [ ]:
# Imports

# Manipulação de dados
import os
import re
import sys
import time
import string
import pickle
import requests
import datetime
import numpy as np
import pandas as pd

# Visualização de dados
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

# Processamento de Linguagem Natural
import spacy
from spacy.symbols import amod
from collections import Counter

# Web Scraping
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Topic Modeling
import sklearn
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF

# Análise de Sentimentos
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Definimos o parâmetro abaixo para evitar o erro:
# RecursionError: maximum recursion depth exceeded
# Ao gravar os resultados em disco
sys.setrecursionlimit(10000)

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

## Parte 1 - Web Scraping

Vamos fazer web scraping dos discursos do prefeito da cidade New York, Bill de Blasio, e do Governador do Estado de New York, Andrew Cuomo. 

Nosso objetivo é recuperar as transcrições de seus discursos oficiais em 2020. Como os sites de onde faremos web scraping são atualizados com discursos frequentemente, esse código produzirá um conjunto diferente de discursos (ou pode não funcionar) cada vez que for executado.

E se algo mudar? Compreenda as mudanças e ajuste o código!!!

Leia e estude o código abaixo. Seu aprendizado, também depende de você!

In [ ]:
# Define o chromedriver (semelhante a um dos Labs anteriores)
# https://chromedriver.chromium.org/downloads
chromedriver = "./chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

### Web Scraping dos Discursos do Prefeito Bill de Blasio

Faremos Web Scraping das transcrições de discurso do prefeito Bill de Blasio. Começaremos raspando as URLs da página principal de transcrições e depois raspando o conteúdo de cada URL. Ou seja, "Web Scraping do Web Scraping".

In [ ]:
# Para armazenar a lista de urls
lista_urls = []

In [ ]:
# Loop para criar a lista de urls para fazer web scraping 
for i in range(1, 41):
    full_url = 'https://www1.nyc.gov/office-of-the-mayor/news.page' + '#page-' + str(i)
    lista_urls.append(full_url)

In [ ]:
# Visualiza a lista
lista_urls

Vamos criar uma função para fazer web scraping.  Altere a variável contador para capturar dados de mais ou menos urls (isso afeta o tempo total de execução do processo de web scraping).

In [ ]:
# Contador
contador = 10

In [ ]:
# Função para web scraping
def scraping_urls(urls):
    
    # Define o driver
    driver = webdriver.Chrome("./chromedriver")
    
    # Lista para o resultado
    soup_list = []
    
    # Contador
    count = 0
    
    # Loop pelas urls
    for i in urls:
        if count < contador:
            driver.get(i)
            driver.refresh()
            time.sleep(5)
            soup_list.append(BeautifulSoup(driver.page_source, 'html.parser'))
        count += 1
    driver.close()
    return soup_list

In [ ]:
# Scraping do código fonte de cada página dos principais discursos
soups = scraping_urls(lista_urls)

In [ ]:
# Visualiza
soups

A função abaixo vai extrair os links do codigo fonte das páginas html. Nesses links encontraremos o texto (transcrição) dos discursos. Usamos aqui expressões regulares em Python, conforme estudado em diversos cursos da DSA.

In [ ]:
# Função para extrair os links de cada url do código fonte
def extrai_links_prefeito(soup_object):
    links_list = []
    for s in soup_object:
        links = s.find_all('a', {'href': re.compile(r'transcript')})
        for i in links:
            link1 = str(i).replace('"', '')
            if re.search('=(.+)>T', link1) is not None:
                link = re.search('=(.+)>T', link1).group(1)
            else:
                continue
            full_link = 'https://www1.nyc.gov' + link
            links_list.append(full_link)
    return links_list

In [ ]:
# Extrai os links para cada página de transcrições de fala
link_list = extrai_links_prefeito(soups)

In [ ]:
# Salvamos os links em disco
with open('dados/db_links_prefeito.pickle', 'wb') as to_write:
    pickle.dump(link_list, to_write)

In [ ]:
# Função para extrair o texto do discurso politico do código fonte
def extrai_discurso_prefeito(urls):
    driver = webdriver.Chrome(chromedriver)
    doc_source = []
    for i in urls:
        driver.get(i)
        time.sleep(5)
        doc_source.append(BeautifulSoup(driver.page_source, 'html.parser'))
    driver.close()
    return doc_source

In [ ]:
# Extrai a transcrição de fala para cada discurso
# Acompanhe a extração em tempo real na outra janela do seu navegador que será aberta!
discursos = extrai_discurso_prefeito(link_list)

In [ ]:
# Salva os discursos extraídos
with open('dados/db_discurso_prefeito.pickle', 'wb') as to_write:
    pickle.dump(discursos, to_write)

### Web Scraping dos Discursos do Governador Andrew Cuomo

O procedimento aqui é similar ao anterior, sendo que algumas páginas são diferentes e teremos que tratar isso em mais detalhes. Leia e estude o código abaixo. Seu aprendizado, também depende de você!

In [ ]:
# Contador (altere para capturar mais ou menos dados)
contador = 10

In [ ]:
# URL com as páginas dos discursos
url = 'https://www.governor.ny.gov/keywords/media'

In [ ]:
# Função para scraping ds urls
def scraping_urls(urls):
    
    # Carrega o driver e extrai o códgo html
    driver = webdriver.Chrome(chromedriver)
    soup_list = []
    driver.get(url)
    driver.refresh()
    time.sleep(5)
    soup_list.append(BeautifulSoup(driver.page_source, 'html.parser'))
    pages = list(range(2,9)) + ([4] * 12) 
    count = 0
    
    # Loop pelas páginas
    for i in pages:
        if count < contador:
            path = '//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(i) + ']'
            driver.find_element_by_xpath(path).click()
            time.sleep(5)
            soup_list.append(BeautifulSoup(driver.page_source, 'html.parser'))
        count += 1
    driver.close()
    return soup_list

In [ ]:
# Scraping do código fonte das páginas
sources = scraping_urls(url)

In [ ]:
# Visualiza
sources

In [ ]:
# Função para extrair os links
def extrai_links_gov(soup_object):
    links_list = []
    for s in soup_object:
        links = s.find_all('a', {'href': re.compile(r'transcript')})
        for i in links:
            link1 = str(i).replace('"', '')
            if re.search('=(.+)>\n', link1) is not None:
                link = re.search('=(.+)>\n', link1).group(1)
            else:
                continue
            full_link = 'https://www.governor.ny.gov' + link
            links_list.append(full_link)
    return list(set(links_list))

In [ ]:
# Extrair os links de cada código fonte
link_list = extrai_links_gov(sources)

In [ ]:
# Salva os links de cada página
with open('dados/db_links_governador.pickle', 'wb') as to_write:
    pickle.dump(link_list, to_write)

In [ ]:
# Função para extrair os discursos
def extrai_discurso_gov(urls):
    driver = webdriver.Chrome(chromedriver)
    doc_source = []
    for i in urls:
        driver.get(i)
        time.sleep(5)
        doc_source.append(BeautifulSoup(driver.page_source, 'html.parser'))
    driver.close()
    return doc_source

In [ ]:
# Extrai os discursos
discursos = extrai_discurso_gov(link_list)

In [ ]:
# Salva o discurso
with open('dados/db_discurso_governador.pickle', 'wb') as to_write:
    pickle.dump(discursos, to_write)

## Limpeza de Dados

Agora que extraímos a fonte de cada página de discursos, precisamos extrair o texto do discurso. 

Infelizmente, não existe um método abrangente e facilmente digerível para obter um discurso limpo e na maioria das vezes esse é um trabalho de tentativa e erro. Experimentamos alternativas até encontrar uma que funcione para o que precisamos, como o que você encontra abaixo.

In [ ]:
# Abre o banco de dados
with open('dados/db_links_prefeito.pickle', 'rb') as read_file:
    db_links_prefeito = pickle.load(read_file) 

In [ ]:
# Abre o banco de dados
with open('dados/db_discurso_prefeito.pickle', 'rb') as read_file:
    db_discurso_prefeito = pickle.load(read_file)

In [ ]:
# Função para extrair o texto
def extrai_texto(source_object):
    text_list = []
    for s in source_object:
        text = s.find_all('p')
        text_list.append(text)
    return text_list

In [ ]:
# Lista para o texto
discurso_length = []

In [ ]:
# Loop para extração do texto
for i in extrai_texto(db_discurso_prefeito):
    discurso_length.append(len(i))

In [ ]:
# Converte para array numpy
discurso_length_array = np.array(discurso_length)

In [ ]:
# Extrai os discursos
primeiros_discursos = extrai_texto(db_discurso_prefeito)

In [ ]:
# Estes são os índices que precisam ser re-extraídos com get_text2
# Isso é necessário pois as páginas tem diferenças entre elas
indices = list(np.argwhere(discurso_length_array == 1).flatten())

In [ ]:
# Função para nova extração do texto
def extrai_texto2(source_object):
    text_list = []
    for s in source_object:
        text = s.find('p').parent
        text_list.append(text)
    return text_list

In [ ]:
# Loop para extrair discursos com texto mais claro
melhores_discursos = []
for i in indices:
    melhores_discursos.append(extrai_texto2(db_discurso_prefeito[i]))

In [ ]:
# Substituir os primeiros_discursos por melhores_discursos
for (indices, melhores_discursos) in zip(indices, melhores_discursos):
    primeiros_discursos[indices] = melhores_discursos

In [ ]:
# Função para remover tags html
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
# Função para limpeza dos dados
def limpa_dados(transcript_list, link_list):
    date = []
    text = []
    for i in transcript_list:
        cleaned = remove_html_tags(str(i))
        if re.search('\[\\n(.+)\\n(.+)\]', cleaned) is not None and re.search('\[\\n(.+)\\n(.+)\]', cleaned) is not None:
            date_clean = re.search('\[\\n(.+)\\n(.+)\]', cleaned).group(1)
            date.append(date_clean)
            text_clean = re.search('\[\\n(.+)\\n(.+)\]', cleaned).group(2)
            text.append(text_clean)
        else:
            date_clean = re.search('\\n\\n(.+20.{2})\\n(.+)\\n\\ufeff', cleaned).group(1)
            date.append(date_clean)
            text_clean = re.search('\\n\\n(.+20.{2})\\n(.+)\\n\\ufeff', cleaned).group(2)
            text.append(text_clean)
    date = pd.to_datetime(date)
    df = pd.DataFrame([date, link_list, text]).T
    df.columns = ['date', 'link', 'text']
    return df

In [ ]:
# Limpeza
dados_limpos = limpa_dados(primeiros_discursos, db_links_prefeito)

In [ ]:
# Função para extrair o discurso usando padrões de expressões regulares
def extrai_texto_discurso(transcript):
    if len(re.findall('sio:([^:]+)|yor:([^:]+)', str(transcript))) > 0:
        return str(re.findall('sio:([^:]+)|yor:([^:]+)', str(transcript)))
    else:
        return ''

In [ ]:
# Padrão para remover
toremove = "'\\,\"\[\(\]\)-–"

In [ ]:
# Funções lambda (anônimas) para extrair caracteres e padrões indesejados dos dados
punc_lower = lambda x: re.sub('[%s]' % re.escape(toremove), '', x.lower())
remove_xa0 = lambda x: x.replace('xa0', '')
remove_space = lambda x: x.replace('  ', ' ')

In [ ]:
# Aplica a limpeza
dados_limpos['monologue'] = dados_limpos['text'].map(extrai_texto_discurso).map(punc_lower).map(remove_xa0).map(remove_space)

In [ ]:
# Incluímos uma coluna indicando que este é um discurso do prefeito deBlasio
dados_limpos.insert(0, 'speaker', 'de blasio')

In [ ]:
# Salvamos os discursos agora limpos
with open('dados/db_discurso_prefeito_limpo.pickle', 'wb') as to_write:
     pickle.dump(dados_limpos, to_write)

Vamos repetir o processo para os discursos do Governador, com alguns pequenos ajustes.

In [ ]:
# Abre o banco de dados
with open('dados/db_links_governador.pickle', 'rb') as read_file:
    db_links_governador = pickle.load(read_file) 

In [ ]:
# Abre o banco de dados
with open('dados/db_discurso_governador.pickle', 'rb') as read_file:
    db_discurso_governador = pickle.load(read_file)

In [ ]:
# Função para obter a data (usaremos isso mais tarde)
def get_date(source_object):
    date_list = []
    for i in source_object:
        date = i.find('div', class_="published-date").text
        date_clean = re.search('\\n\\n(.+20.{2})', date).group(1).strip()
        date_list.append(date_clean)
    return date_list

In [ ]:
# Extrai a data
cuomo_date = get_date(db_discurso_governador)

In [ ]:
# Função para extrair o texto
def get_text(source_object):
    text_list = []
    for s in source_object:
        text = s.find('div', class_='field field--name-field-body field--type-text-long field--label-hidden')
        text_list.append(text)
    return text_list

In [ ]:
# Extrai o texto com uma primeira passada pelos dados
cuomo_step1 = get_text(db_discurso_governador)

In [ ]:
# Função para limpeza dos dados
def clean_cuomo(transcript_list):
    clean_transcripts = []
    for idx, i in enumerate(transcript_list):
        cleaned = remove_html_tags(str(i))
        if re.search('below:(.+)', cleaned) is not None:
            text_clean = re.search('below:(.+)', cleaned).group(1)
            clean_transcripts.append(text_clean)
        elif re.search('here.(.+)', cleaned) is not None:
            text_clean = re.search('here.(.+)', cleaned).group(1)
            clean_transcripts.append(text_clean)
    return clean_transcripts

In [ ]:
# Limpa os dados
cuomo_step2 = clean_cuomo(cuomo_step1)

In [ ]:
# Vamos visualizar o que extraímos
test_len = []
for i in cuomo_step2:
    test_len.append(len(i))
test_len_array = np.array(test_len)

In [ ]:
# Exemplo de texto extraído
cuomo_step2[68]

In [ ]:
# Exemplo de texto extraído
cuomo_step2[99]

In [ ]:
# Para liberar memória, deletamos objetos que não precisamos mais
delete_index = test_len_array.argsort()[0]
del cuomo_step2[delete_index]
del db_links_governador[delete_index]
del cuomo_date[delete_index]

In [ ]:
# Convertemos a data para o tipo datetime
cuomo_date = pd.to_datetime(cuomo_date)

In [ ]:
# Criamos o dataframe com dados limpos
cuomo_clean = pd.DataFrame([cuomo_date, db_links_governador, cuomo_step2]).T
cuomo_clean.columns = ['date', 'links', 'text']

In [ ]:
# Visualiza
cuomo_clean.head()

In [ ]:
# Salvamos os dicusros
with open('dados/db_discurso_governador_limpo.pickle', 'wb') as to_write:
    pickle.dump(cuomo_clean, to_write)

Usaremos três funções diferentes com expressões regulares diferentes para extrair apenas as partes do discurso em que o Governador estava falando (não queremos comentários de outras pessoas). Para nossa sorte, temos essas informação na página de cada discurso.

In [ ]:
# Função para extrair discurso
def extrai_discurso_gov1(transcript):
    if len(re.findall('Cuomo:([^:]+)', str(transcript))) > 0:
        return str(re.findall('Cuomo:([^:]+)', str(transcript)))
    else:
        return str(transcript)

In [ ]:
# Função para extrair discurso
def extrai_discurso_gov2(transcript):
    if len(re.findall('Cuomo:(.*?)[A-Z][a-z]+:', str(transcript))) > 0:
        return str(re.findall('Cuomo:(.*?)[A-Z][a-z]+:', str(transcript)))
    else:
        return str(transcript)

In [ ]:
# Função para extrair discurso
def extrai_discurso_gov3(transcript):
    if len(re.findall('^(.*?)[A-Z][a-z]+:', str(transcript))) > 0:
        return str(re.findall('^(.*?)[A-Z][a-z]+:', str(transcript)))
    else:
        return str(transcript)

In [ ]:
# Limpeza dos dados
cuomo_clean['monologue'] = cuomo_clean['text'].map(extrai_discurso_gov1)

In [ ]:
# Limpeza dos dados
cuomo_clean['monologue2'] = cuomo_clean['text'].map(extrai_discurso_gov2)

In [ ]:
# Use a terceira função apenas se os dois primeiros não capturarem a transcrição correta
def extract_m3(col1, col2):
    if (len(col1)-len(col2))/len(col2) > 10:
        return extrai_discurso_gov3(col1)
    else:
        return ''

In [ ]:
# Limpeza dos dados
cuomo_clean['monologue3'] = cuomo_clean.apply(lambda x: extract_m3(x.text, x.monologue), axis = 1)

In [ ]:
# Função para comparação
def compara(col1, col2):
    if (len(col1) - len(col2)) / len(col2) > 1:
        return col1
    else:
        return col2

In [ ]:
# Texto final com duas extrações
cuomo_clean['final_text'] = cuomo_clean.apply(lambda x: compara(x.monologue, x.monologue2), axis = 1)

In [ ]:
# Texto final com a terceira extração
cuomo_clean['final_text2'] = cuomo_clean.monologue3 + cuomo_clean.final_text

In [ ]:
# Removemos caracteres indesejados
remove_sxa0 = lambda x: x.replace('\xa0', '')

In [ ]:
# Limpeza final
cuomo_clean['final_clean'] = cuomo_clean['final_text2'].map(punc_lower).map(remove_xa0).map(remove_sxa0).map(remove_space)

In [ ]:
# Limpeza final
cuomo_final = cuomo_clean.loc[:, ['date', 'links', 'text', 'final_clean']]
cuomo_final.columns = ['date', 'link', 'text', 'monologue']

In [ ]:
# Incluímos uma coluna indicando que este é um discurso do Governador Cuomo
cuomo_final.insert(0, 'speaker', 'cuomo')

In [ ]:
# Visualiza
cuomo_final.head()

In [ ]:
# Salva em disco
with open('dados/db_discurso_governador_limpo_final.pickle', 'wb') as to_write:
    pickle.dump(cuomo_final, to_write)

Abrimos os bancos de dados já limpos para concatená-los no mesmo dataframe.

In [ ]:
with open('dados/db_discurso_prefeito_limpo.pickle','rb') as read_file:
    deblasio = pickle.load(read_file)  

In [ ]:
with open('dados/db_discurso_governador_limpo_final.pickle','rb') as read_file:
    cuomo = pickle.load(read_file)

In [ ]:
# Concatena os databases dos discursos
db_final = pd.concat([cuomo, deblasio], axis = 0).reset_index(drop = True)

In [ ]:
# Verificamos a proporção de discursos para cada político
db_final['speaker'].value_counts()

In [ ]:
# Grava o database final
with open('dados/db_final.pickle', 'wb') as to_write:
     pickle.dump(db_final, to_write)

Se necessário retorne e colete mais dados.

## Parte 3 - Topic Modeling

Modelagem de Tópicos é uma forma de mineração de texto, uma forma de identificar padrões. Construindo um corpus e executando uma ferramenta que gera grupos de palavras a respeito do corpus distribuídas em “tópicos”. 

Modelagem de tópicos é um método para achar e traçar clusters de palavras (chamado “tópicos” de forma abreviada) em grandes conjuntos de texto.

Aqui tem uma definição completa sobre o tema:

http://journalofdigitalhumanities.org/2-1/topic-modeling-a-basic-introduction-by-megan-r-brett/#topic-modeling-a-basic-introduction-by-megan-r-brett-n-1

In [ ]:
# Download do modelo de linguagem do spacy
!python -m spacy download en_core_web_sm

In [ ]:
# Para usar o spaCy, precisamos referenciar o modelo de linguagem
sp = spacy.load("en_core_web_sm")

In [ ]:
# Carregamos o banco de dados
with open('dados/db_final.pickle','rb') as read_file:
    db_final = pickle.load(read_file)

In [ ]:
# Para o Processamento de Linguagem Natural criamos alguns padrões para extração no texto dos discursos
alphanumeric = lambda x: re.sub('\w*\d\w*', '', x)
punc = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
remove_space = lambda x: x.replace('  ', ' ')

In [ ]:
# Aplicamos as funções anteriores ao database final
db_final['for_spacy'] = (db_final['monologue']
                         .map(alphanumeric)
                         .map(punc)
                         .map(remove_space))

In [ ]:
# Agora aplicamos o modelo de linguagem (sp) do SpaCy
db_final['spacy_monologue'] = db_final['for_spacy'].map(lambda x: sp(x))

In [ ]:
# Visualiza
db_final.head()

Lematização é o processo de extrair o lema de cada palavra, tarefa fundamental em PLN. Não precisamos da palavra inteira, somente do seu lema.

In [ ]:
# Lematização
db_final['lemmatized'] = (db_final['spacy_monologue']
                          .map(lambda x: [' '.join(word.lemma_ 
                                                   if word.lemma_ != '-PRON-' 
                                                   else word.text for word in x)][0]))

In [ ]:
# Visualiza
db_final.head()

Aqui nós criamos uma lista de stop words palaras que não são relevantes ou representam os nomes dos entrevistadores, que também não são necessárias para esta análise.

In [ ]:
# Lista de stop words
lista_stop_words = (text
                    .ENGLISH_STOP_WORDS
                    .union(['lehrer', 'brian', 'darden', 'moderator', 'alan', 'howard', 'wolf', 'blitzer', 
                            'errol', 'louis', 'alisyn', 'chris', 'camerota', 'dan', 'mannarino','john', 'berman', 
                            'savannah', 'guthrie', 'hoda']))

In [ ]:
# Configurando max_df para 0,5 e min_df = 2, porque estes forneceram os melhores tópicos
cv1 = CountVectorizer(stop_words = lista_stop_words, max_df = 0.5, min_df = 2)

In [ ]:
# Cria a matriz de documentos, que basicamente contém palavras em representações numéricas
docterm_matrix = cv1.fit_transform(db_final.loc[:, 'lemmatized'])

In [ ]:
# Labels
doc_label = ['Document' + str(t) for t in range(len(db_final.loc[:, 'lemmatized']))]

In [ ]:
# Matriz pronta
pd.DataFrame(docterm_matrix.toarray(), index = doc_label, columns = cv1.get_feature_names()).iloc[:10, 630:650]

Usaremos Non-Negative Matrix Factorization (NMF) para preencher a matriz com os tópicos por documento.

O código abaixo usa o NMF para 12 tópicos, mas tentamos vários hiperparâmetros diferentes (incluindo a configuração de max_df e min_df no count_vectorizer) antes de escolher o 12, que fornecia os tópicos mais claros e distintos.

In [ ]:
# Cria e treina o modelo
nmf_cv = NMF(12)
nmf_topics1 = nmf_cv.fit_transform(docterm_matrix)

In [ ]:
# Dataframe de tópicos
topicword_cv1 = pd.DataFrame(nmf_cv.components_.round(3),
                            index = ['topic0', 'topic1', 'topic2', 'topic3',
                                     'topic4', 'topic5', 'topic6', 'topic7',
                                     'topic8', 'topic9', 'topic10', 'topic11'],
                            columns = cv1.get_feature_names()) 

Criamos um dataframe com os 12 tópicos nas linhas e os termos nas colunas. Os valores no dataframe descrevem como o termo se relaciona com o tópico, com valores mais altos indicando um relacionamento mais forte.

In [ ]:
# Matriz
topicword_cv1.iloc[:, 1790:1820]

In [ ]:
# Função para encontrar as palavras mais importantes por tópico
def top_words_per_topic(model, terms, topic_names = None):
    for ix, topic in enumerate(nmf_cv.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTópico ", ix)
        else:
            print("\nTópico '",topic_names[ix],"'")
        print(", ".join([cv1.get_feature_names()[i] for i in topic.argsort()[:-10 - 1:-1]]))

In [ ]:
# Buscamos os speakers (políticos)
db_final_docs_topics = db_final.loc[:, ['speaker']]

In [ ]:
# Visualiza
db_final_docs_topics

In [ ]:
# Buscamos os tópicos mais relevantes
db_final_docs_topics['topics12'] = nmf_topics1.argmax(axis = 1) 

Tendo em vista esses tópicos, vamos entender como os palestrantes se envolveram com cada tópico. Para isso, analisamos as contagens gerais por tópico e depois dividimos por orador.

In [ ]:
# Tópicos por político
speaker_topics = pd.DataFrame(db_final_docs_topics.groupby(['topics12']).speaker.value_counts())
speaker_topics.columns = ['count']
speaker_topics.reset_index()
speaker_topics_pivot = speaker_topics.reset_index().pivot_table(index = 'topics12', columns = 'speaker', values = 'count', fill_value = 0).reset_index(drop = True)
speaker_topics_pivot['total'] = speaker_topics_pivot['cuomo'] + speaker_topics_pivot['de blasio']
speaker_topics_pivot.sort_values('total', inplace = True)

Visualizamos o resultado!

In [ ]:
# Plot
plt.figure(figsize = (14, 8))
names = list(set(speaker_topics_pivot.index))
plt.title('Tópicos Mais Abordados nos Discursos')
plt.xlabel('Total de Documentos por Tópico')
plt.xlim(0, 80)
plt.barh(names, speaker_topics_pivot['total'], color = 'magenta', edgecolor = 'white', height = barWidth)
plt.yticks(range(0, 12), ['Neighborhood Impact', 'Covid Mechanics', 'Healthcare', 'Education/School', 
                         'Neighborhood Resilience', 'Homelessness', 'DOH Communication', 'New Yorkers',
                         'Hate Crime', 'Hospital Needs', 'Hospital Status', 'Reopening Metrics'])
plt.tight_layout()

O tópico que aparece com mais frequência refere-se a métricas para reabertura. Isso inclui termos como taxa, vírus, reabertura, infecção, negócios, infecção e hospitalização. Isso faz sentido, já que a cidade de New York está em processo de reabertura da economia no momento que executamos este Jupyter Notebook.

In [ ]:
# Plot
plt.figure(figsize = (14, 8))
names = list(set(speaker_topics_pivot.index))
plt.title('Politicos Cobrindo Diferentes Tópicos')
plt.xlabel('Total de Documentos por Tópico')
plt.xlim(0, 80)
plt.barh(names, speaker_topics_pivot['cuomo'], color = '#77a9cf', edgecolor = 'white', height = barWidth, label = 'Cuomo')
plt.barh(names, speaker_topics_pivot['de blasio'], left = speaker_topics_pivot['cuomo'], color = '#df8a62', edgecolor = 'white', height = barWidth, label = 'de Blasio')
plt.yticks(range(0, 12), ['Neighborhood Impact', 'Covid Mechanics', 'Healthcare', 'Education/School', 
                         'Neighborhood Resilience', 'Homelessness', 'DOH Communication', 'New Yorkers',
                         'Hate Crime', 'Hospital Needs', 'Hospital Status', 'Reopening Metrics'])
plt.legend()
plt.tight_layout()

A reabertura é puramente um tópico de Cuomo. Cuomo também iniciou muitos de seus discursos, fornecendo estatísticas relevantes para a reabertura. O Prefeito de Blasio domina o tópico das necessidades hospitalares, enquanto Cuomo discute o status dos hospitais. Eles descrevem coisas diferentes, nas quais os discursos de Cuomo tendem a envolver atualizações sobre ventiladores, camas e equipamentos disponíveis, e de Blasio aborda o que os hospitais de New York estão perdendo: pessoal, suprimentos e ventiladores. Ambos estão cobrindo hospitais de maneiras ligeiramente diferentes.

O Prefeito de Blasio se concentra no atendimento público universal e disponível, enquanto Cuomo se concentra no tratamento apropriado e intensivo. A análise de partes do discurso destaca as raízes progressivas de Blasio, com seu foco nos nova-iorquinos da classe trabalhadora e a necessidade de cuidados de saúde públicos universais.

## Parte 4 - Análise de Sentimentos

Esta é a última parte do trabalho.

Usaremos a função SentimentIntensityAnalyzer e alimentamos os discursos para recuperar as pontuações de sentimentos. O pacote VADER retorna uma pontuação negativa, neutra, positiva e composta, que pode ser plotada ao longo do tempo para entender como os sentimentos mudam ao longo do tempo.

In [ ]:
# Carrega o dataset
with open('dados/db_final.pickle','rb') as read_file:
    db_final = pickle.load(read_file)

In [ ]:
# Cria o analisador de sentimentos
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Lista de scores
scores = []

In [ ]:
# Calcula os scores de sentimentos
for i in db_final['monologue']:
    scores.append(analyzer.polarity_scores(i))

In [ ]:
# Concatenamos o resultado 
db_final_docs_sentiment = pd.concat([db_final.loc[:, ['speaker', 'date']], pd.DataFrame(scores)], axis = 1)

Agora calculamos médias de sentimentos para cada orador, observando a pontuação composta e a positiva. É como criar uam janela deslizante em análise de série temporal.

In [ ]:
# Divide os dados por político
cuomo_roll = db_final_docs_sentiment[db_final_docs_sentiment.speaker == 'cuomo'].sort_values(by = 'date')
de_blasio_roll = db_final_docs_sentiment[db_final_docs_sentiment.speaker == 'de blasio'].sort_values(by = 'date')

In [ ]:
# Médias móveis do Governador
cuomo_roll['cp_roll_avg'] = cuomo_roll.compound.rolling(window = 7).mean()
cuomo_roll['pos_roll_avg'] = cuomo_roll.pos.rolling(window = 7).mean()

In [ ]:
# Médias móveis do Prefeito
de_blasio_roll['cp_roll_avg'] = de_blasio_roll.compound.rolling(window = 7).mean()
de_blasio_roll['pos_roll_avg'] = de_blasio_roll.pos.rolling(window = 7).mean()

In [ ]:
# Combinamos as médias móveis em um único dataframe
combined_roll = pd.concat([cuomo_roll, de_blasio_roll]).sort_values('date').reset_index(drop = True)

In [ ]:
# Visualiza
combined_roll.head(10)

Perfeito. Temos uma série temporal de discursos. Várias análises e previsões poderiam ser feitas aqui e deixaremos isso com você. Vamos apenas plotar o resultado final.

Lembre-se que alunos das Formações DSA tem acesso ao curso bônus gratuito de Análise de Séries Temporais.

In [ ]:
# Plot do Score Composto
fig = plt.figure(figsize = (15, 8))
ax = plt.axes()
sns.lineplot(x = 'date', y = 'cp_roll_avg', hue = 'speaker', data = combined_roll)
plt.title('Score Composto 7-Dias Rolling Average')
plt.xlabel('Data do Discurso')
plt.ylabel('Score Composto')
plt.ylim((-0.6, 1.1))
plt.axhline(y = 0, color = '#77a9cf', linestyle = ':')
plt.axvline(x = datetime.date(2020, 7, 1), color = 'black', linestyle = 'dashed')
plt.xticks(rotation = 30)
plt.tight_layout()

Olhando para o gráfico da pontuação composta, percebemos que o Governador Cuomo teve pontuação baixa (indicando discurso com tom mais negativo) em Junho/2020, melhorando sua pontuação em Julho. O Prefeito de Blasio manteve seus discursos com tom positivo de forma quase constante.

In [ ]:
# Plot do Score Positivo
fig = plt.figure(figsize = (15, 8))
ax = plt.axes()
sns.lineplot(x = 'date', y = 'pos_roll_avg', hue = 'speaker', data = combined_roll)
plt.title('Score Positivo 7-Dias Rolling Average')
plt.xlabel('Data do Discurso')
plt.ylabel('Score Positivo')
plt.axvline(x = datetime.date(2020, 7, 1), color = 'black', linestyle = 'dashed')
plt.xticks(rotation = 30);
plt.tight_layout()

Olhando para o gráfico de pontuação positiva, a média móvel de 7 dias também é mais alta para o Prefeito de Blasio do que para o Governador Cuomo. 

Esse trabalho ainda permite diversas análises, modelagem preditiva ou mesmo comparações com outros eventos. Experimente modificar o projeto e continuar o processo de análise.

# Fim